In [82]:
import requests
import urllib.request
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import pandas as pd
from itertools import combinations

In [21]:
def get_page(url):

    string = ''
    
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content, "html.parser")
    
    for body_child in soup.body.children:
        if isinstance(body_child, NavigableString):
            continue
        if isinstance(body_child, Tag):
                lds = body_child.find_all('li')
                #tds = tr.find_all('td')
                for val in lds:
                    string = string + ' ' + val.text
    return string

In [49]:
page = get_page(url)

In [79]:
pol_ang = {'ą':'a', 'ć':'c', 'ę':'e', 'ł':'l', 'ń':'n', 'ó':'o', 'ś':'s', 'ź':'z', 'ż':'z',
          'Ą':'A', 'Ć':'c', 'Ę':'E', 'Ł':'L', 'Ń':'N', 'Ó':'O', 'Ś':'S', 'Ź':'Z', 'Ż':'Z'}

string = ''
for l in page:
    if l in pol_ang.keys():
        l = pol_ang[l]
    string = string + l
    

sub = re.sub(r"\xa0+", " ", string)
all_pubs = re.findall(r"(?s)(?:[a-zA-Z]\.)[^\d]*?(?=, \w[^\.])+", sub)
all_pubs = [pub.split(', ') for pub in all_pubs]

temp = []
for pub in all_pubs:
    t1 = [name.split('. ') for name in pub]
    temp.append(t1)

all_pubs = []
for pub in temp:
    p1 = []
    for name in pub:
        p1.append([name[0], name[-1]])
        
    all_pubs.append(p1)
        
co_work = []
for pub in all_pubs:
    co_work.append([name[0]+'.'+name[1] for name in pub])


[['M.Sonner',
  'A.Sitek',
  'L.Janker',
  'D.Rudolph',
  'D.Ruhstorfer',
  'M.Döblinger',
  'A.Manolescu',
  'G.Abstreiter',
  'J.Finley',
  'A.Wixforth',
  'G.Koblmüller',
  'H.Krenner'],
 ['S.Chen',
  'Z.Lu',
  'T.Goldstein',
  'J.Tong',
  'A.Chaves',
  'J.Kunstmann',
  'L.Cavalcante',
  'T.Wozniak',
  'G.Seifert',
  'D.Reichman',
  'T.Taniguchi',
  'K.Watanabe',
  'D.Smirnov',
  'J.Yan'],
 ['J.Jadczak',
  'L.Bryja',
  'J.Kutrowska-Girzycka',
  'P.Kapuscinski',
  'M.Bieniek',
  'Y.Huang',
  'P.Hawrylak'],
 ['M.Mierzejewski', 'P.Prelovšek', 'J.Bonča'],
 ['J.Herbrych',
  'J.Heverhagen',
  'N.Patel',
  'G.Alvarez',
  'M.Daghofer',
  'A.Moreo',
  'E.Dagotto'],
 ['S.Bednarek', 'J.Pawlowski', 'M.Gorski', 'G.Skowron'],
 ['G.Pawlik', 'T.Wysoczanski', 'A.Mitus'],
 ['W.Radosz', 'R.Orlik', 'G.Pawlik', 'A.Mitus'],
 ['M.Sroda', 'P.Prelovšek', 'M.Mierzejewski'],
 ['B.Jaworowski', 'N.Regnault', 'Z.Liu'],
 ['K.Gawarecki', 'M.Krzykowski'],
 ['B.Kusmierz', 'A.Wojs', 'G.Sreejith'],
 ['M.Kozarzewski', 

In [80]:
ph_emps = pd.read_csv('ph_emps.csv')
sh_names = [name[0]+'.'+surname for name, surname in zip(ph_emps["name"], ph_emps["surname"])]
f_names = [name+' '+surname for name, surname in zip(ph_emps["name"], ph_emps["surname"])]
emp_d = dict(zip(sh_names, f_names))

In [89]:
all_pubs = []
for pub in co_work:
    all_pubs.append([name for name in pub if name in emp_d.keys()])
    
edges = [list(combinations(item, 2)) for item in all_pubs if len(item) > 1]
edges = [edge for pub in edges for edge in pub]
edges = [(emp_d[edge[0]], emp_d[edge[1]]) for edge in edges]

pd.DataFrame(edges, columns=["node1", "node2"]).to_csv("kft_edges.csv", index=False)